In [11]:
import torch
import sys
sys.path.append("/home3/ebrahim/neural_seq_decoder/src")
from neural_decoder.bit import BiT_Phoneme
from collections import OrderedDict

In [24]:
model_path = "/home3/skaasyap/willett/outputs/enc_7_dec_3_no_cosine_4_17/save_best.pth"
pretrained_state = torch.load(f'{model_path}', map_location='cpu')['model_state_dict']
# Step 2: Extract the transformer weights from encoder.transformer
transformer_weights = OrderedDict()
prefix = "encoder.transformer."
for k, v in pretrained_state.items():
    if k.startswith(prefix):
        new_key = k[len(prefix):]  # strip the prefix
        transformer_weights[new_key] = v

In [18]:
model = BiT_Phoneme(patch_size=(5,256), dim=384, depth=7, heads=6, mlp_dim_ratio=4, dim_head=64, dropout=0.4, input_dropout=0.2, look_ahead=0, nDays=24, gaussianSmoothWidth=2.0, 
            nClasses=40, T5_style_pos=True, max_mask_pct=0.05, num_masks=20, mae_mode=False)

In [19]:
model.transformer.load_state_dict(transformer_weights, strict=False)

<All keys matched successfully>

In [20]:
model.mask_token

Parameter containing:
tensor([-0.9647, -0.4466,  0.1040,  ...,  0.7947,  1.3381,  1.7217],
       requires_grad=True)

In [21]:
with torch.no_grad():
    model.mask_token.copy_(pretrained_state["encoder.mask_token"])

In [22]:
model.mask_token

Parameter containing:
tensor([-0.3003,  0.3955, -0.5775,  ..., -0.0681, -0.9625,  0.5753],
       requires_grad=True)